
## Branching out with `if`, _pattern matching_ and recursion.

One of the most important concepts in programming is control flow. This refers to the possibility of the code to handle different execution paths according to conditions that appear when processing data. For example, the user of the vending machine can pay with bills, coins or a credit card, and the software system must be able to accommodate all of these cases.

In most languages ​​there is an `if condition then ... else` statement, which allows code to be managed according to whether the condition is true or false.

However, in F#, we don't have statements but expressions, so the `if` is also evaluated as an expression and must return a value. For example, we can evaluate the function $|x|$:

In [2]:
let absoluteValue x =
    let v = 
        if x < 0 then
            -x 
        else 
            x
    v

printfn "%A" (absoluteValue 4)
printfn "%A" (absoluteValue -5)

4
5


In this example, the result of the evaluation of the `if` expression is stored in the identifier `v` which is the return value of the function. You can write the `if` on a single line:

In [4]:
let absoluteValue2 x =
    let v = if x < 0 then -x else x 
    v
            
printfn "%A" (absoluteValue2 2)
printfn "%A" (absoluteValue2 -8)

2
8


And you don't even need to use `v`, since the conditional is evaluated and its result is what the function returns:

In [6]:
let absoluteValueOneliner x =
    if x < 0 then -x else x 
    
            
printfn "%A" (absoluteValueOneliner 2)
printfn "%A" (absoluteValueOneliner -8)

2
8


The `if` expression is nothing more than a _syntactic sweetener_ for a pattern matching expression:

In [7]:
let absoluteValuePM x =
    match x < 0 with 
    | true -> -x 
    | false -> x 
    
            
printfn "%A" (absoluteValuePM 2)
printfn "%A" (absoluteValuePM -8)

2
8


Indeed, the expression `x < 0` is of type `bool`, which has two possible cases, `true` and `false`.

Also note that both branches of the conditional must evaluate to the same data type, in the example above, `int`.

The language doesn't provide another variant of the `if` conditional, because that's what it's for...


### Pattern Matching

One of the strengths of the language lies in the construction of _pattern matching_. So far we have used it to select the possible cases in a discriminated union. However, _pattern matching_ is more general and, unlike `if`, allows you to create multiple branches. To do this, the keyword `when` is used, as follows:

In [11]:
let votingAge = 16
let buyingOHAge = 18 

let checkAge (age: int) =
    match age with 
    | a when a < votingAge -> printfn "No vota ni puede comprar vino"
    | a when a >= votingAge && a < buyingOHAge -> printfn "Puede votar, pero no comprar vino"
    | a when a >= buyingOHAge -> printfn "Puede votar y comprar vino" 


checkAge 15
checkAge 17
checkAge 50

No vota ni puede comprar vino
Puede votar, pero no comprar vino
Puede votar y comprar vino


In this case, _pattern matching_ allows us to decide between three age options. Note that the `checkAge` function returns the type `unit`. Another example would be:

In [2]:
type WeatherWarning = 
    | Freezing 
    | YellowAlertHot
    | RedAlertHot 
    | NoAlarm 

let emitAlarm warning = 
    match warning with 
    | Freezing -> printfn "%A" "Alarm: Freezing"
    | YellowAlertHot-> printfn "%A" "Alarm: Yellow"
    | RedAlertHot -> printfn "%A" "Alarm: Red"
    | NoAlarm  -> printfn "%A" "It's fine "

let checkTempForWarning temp =
    match temp with 
    | t when t <= 0 -> Freezing 
    | t when t > 33 && t < 39 -> YellowAlertHot
    | t when t >= 39 -> RedAlertHot
    | _ -> NoAlarm     

emitAlarm (checkTempForWarning 43)

checkTempForWarning 37
|> emitAlarm 

-3 
|> checkTempForWarning 
|> emitAlarm 



"Alarm: Red"
"Alarm: Yellow"
"Alarm: Freezing"


In [5]:
checkTempForWarning 43

checkTempForWarning -4

45 
|> checkTempForWarning 


RedAlertHot

In [3]:
type WeatherWarningXtreme = 
    | Freezing 
    | TooCold
    | Superconducting

let checkTempForWarning2 temp =
    match temp with 
    | t when t <= 0 -> Freezing
    | t when t > -200 && t<0 -> TooCold
    | t when t < -200 -> Superconducting
    // | _ -> NoAlarm     

One more example to see how the _pattern matching_ syntax is in the situation in which you want to group cases:

In [15]:
type Animals =
    | Cat
    | Dog 
    | Eagle  

let howManyLegs animal =
    match animal with
    | Animals.Cat | Animals.Dog -> 4  // grouping two cases
    | Animals.Eagle -> 2

printfn "%A" (howManyLegs Animals.Cat)
printfn "%A" (howManyLegs Animals.Dog)
printfn "%A" (howManyLegs Animals.Eagle)


4
4
2


This is possible in the case where the _pattern matching_ is against expressions that are or represent constants.

## Replacing the `for` with recursion

Since the usual `for` loop requires (either explicitly or implicitly) some sort of counter that is being updated, that is, it is a _variable_, it is necessary to have a way to loop without the `for`. For this, recursion is used in all functional languages.

For example, if one has the loop

```c
    for(int i = 0; i < n; i++)
    {
        printf("%d\n",i);
    }
```

It will be replaced by a recursive function:

In [5]:
let printNumbers n = 

    let rec loop i n =
        if i < n then
            printfn "%d" i
            loop (i + 1) n
        else 
            printfn "done"

    loop 0 n

printNumbers 3    

0
1
2
done


The first point to note is that, since in the language there are only expressions, `for` loops are going to be represented by functions, it is no longer possible to have a loop as a set of statements that are executed. In general, loops as statements also imply an application to variables of what happens inside the loop.



Let's look at the classic example of the Fibonacci series:
$$
a_n = a_{n-1} + a_{n-2}
$$
with $a_0=0$ and $a_1=1$.

This example, written in C would be:
```c
int fib(int n) {
    int prev1 = 0, prev2 = 1, curr = 0;
    if (n == 0)
        return prev1;
    else if (n == 1)
        return prev2;
    else {
        for (int i = 2; i <= n; i++) {
            curr = prev1 + prev2;
            prev1 = prev2;
            prev2 = curr;
        }
        return curr;
    }
}
```

The non-recursive version involves saving the state of the previous two values ​​of the string within the function, which is done by using `prev1=0` and `prev2=1`.

The recursive version in C would be
```c
int fib(int n) {
    if (n < 2)
        return n;
    else
        return fib(n-1) + fib(n-2);
}
```

In F# recursion works in a similar way, although _pattern matching_ can be used:

In [18]:
let rec fib n  = 
    match n with
    | 0 | 1 -> n
    | n -> fib (n-1) + fib (n - 2)
    
fib 10 |> printfn "%i" 

55


One might be interested in adding a series of numbers, for example calculating the series
$$
1 + \frac{1}{2} + \frac{1}{3} + \cdots + \frac{1}{n}
$$
up to a certain value of $n$:

In [22]:
let rec sumSeries n =
    match n with
    | 0 -> 0.0
    | n -> 1.0/(float n) + sumSeries(n-1)

printfn "%A" (sumSeries 100)    

5.187377518


In practice, the purest way of programming a recursion is not the most suitable. For example, in the case of the Fibonacci sequence, there are calls to the recursive function that recalculate values ​​that have already been calculated. That is why the method known as _tail recursion_ is frequently used. In this case, the unnecessary call to the recursive function is prevented by introducing new values ​​that carry the information from the previous state of the recursion to the present calculation:

In [ ]:
let fib n =
    let rec loop acc1 acc2 n =
        match n with
        | 0 -> acc1
        | 1 -> acc2
        | _ ->
            loop acc2 (acc1 + acc2) (n - 1)
    loop 0 1 n

In this case the outer function `fib` is not recursive itself, but uses a recursive function inside it, called `loop`. The `acc1` and `acc2` values ​​are in charge of carrying the information of the previous elements of the series necessary to calculate the present element.

> 👀 It's important to remember that F# is a _multiparadigm_ language, so, strictly speaking, it does have a `for` similar to what one knows in other languages. In this particular course we are focusing on the functional paradigm of F#, which is why we **choose** to discard the use of the traditional `for`.